In [89]:
import pandas as pd

In [90]:
df1 = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')

In [91]:
df1

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/5/21,4/6/21,4/7/21,4/8/21,4/9/21,4/10/21,4/11/21,4/12/21,4/13/21,4/14/21
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,...,56717,56779,56873,56943,57019,57144,57160,57242,57364,57492
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,126795,126936,127192,127509,127795,128155,128393,128518,128752,128959
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,117739,117879,118004,118116,118251,118378,118516,118645,118799,118975
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,12286,12328,12363,12409,12456,12497,12545,12581,12614,12641
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,22717,22885,23010,23108,23242,23331,23457,23549,23697,23841
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269,NaN,Vietnam,14.058324,108.277199,0,2,2,2,2,2,...,2637,2648,2659,2668,2683,2692,2693,2705,2714,2733
270,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,...,253922,256461,259133,262017,264395,265897,268132,270856,272767,274690
271,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,...,4881,4975,5047,5133,5233,5276,5357,5446,5507,5582
272,NaN,Zambia,-13.133897,27.849332,0,0,0,0,0,0,...,89009,89071,89386,89592,89783,89918,90029,90064,90218,90389


In [92]:
df1.shape

(274, 453)

In [93]:
countries=df1["Country/Region"].count()
unique_countries=df1["Country/Region"].nunique(dropna = True)
print("There are ",countries ,"entries for the country column while unique no of countries is ",unique_countries)

There are  274 entries for the country column while unique no of countries is  192


This means there are repeated countries due to the seperate divisions for some countries inthe Province/State column like the US and Australia 

In [94]:
df1["Province/State"].nunique(dropna = True)

85

##### Drop the Province/State column, then group all rows by country.

In [95]:
df1=df1.drop('Province/State',axis=1)
df1.shape

(274, 452)

In [96]:
df1["Country/Region"].nunique(dropna = True)

192

In [97]:
df1=df1.groupby(['Country/Region'],as_index=False).sum()

In [98]:
df1.shape

(192, 452)

In [99]:
df1.head()

,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,...,4/5/21,4/6/21,4/7/21,4/8/21,4/9/21,4/10/21,4/11/21,4/12/21,4/13/21,4/14/21
0,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,0,...,56717,56779,56873,56943,57019,57144,57160,57242,57364,57492
1,Albania,41.15330,20.168300,0,0,0,0,0,0,0,...,126795,126936,127192,127509,127795,128155,128393,128518,128752,128959
2,Algeria,28.03390,1.659600,0,0,0,0,0,0,0,...,117739,117879,118004,118116,118251,118378,118516,118645,118799,118975
3,Andorra,42.50630,1.521800,0,0,0,0,0,0,0,...,12286,12328,12363,12409,12456,12497,12545,12581,12614,12641
4,Angola,-11.20270,17.873900,0,0,0,0,0,0,0,...,22717,22885,23010,23108,23242,23331,23457,23549,23697,23841


In [100]:
df1=df1.drop(['Lat','Long'],axis=1)    

In [101]:
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv')

In [102]:
df

,COUNTRY,GDP (BILLIONS),CODE
0,Afghanistan,21.71,AFG
1,Albania,13.40,ALB
2,Algeria,227.80,DZA
3,American Samoa,0.75,ASM
4,Andorra,4.80,AND
...,...,...,...
217,Virgin Islands,5.08,VGB
218,West Bank,6.64,WBG
219,Yemen,45.45,YEM
220,Zambia,25.61,ZMB


In [103]:
df1

,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,...,4/5/21,4/6/21,4/7/21,4/8/21,4/9/21,4/10/21,4/11/21,4/12/21,4/13/21,4/14/21
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,56717,56779,56873,56943,57019,57144,57160,57242,57364,57492
1,Albania,0,0,0,0,0,0,0,0,0,...,126795,126936,127192,127509,127795,128155,128393,128518,128752,128959
2,Algeria,0,0,0,0,0,0,0,0,0,...,117739,117879,118004,118116,118251,118378,118516,118645,118799,118975
3,Andorra,0,0,0,0,0,0,0,0,0,...,12286,12328,12363,12409,12456,12497,12545,12581,12614,12641
4,Angola,0,0,0,0,0,0,0,0,0,...,22717,22885,23010,23108,23242,23331,23457,23549,23697,23841
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,Vietnam,0,2,2,2,2,2,2,2,2,...,2637,2648,2659,2668,2683,2692,2693,2705,2714,2733
188,West Bank and Gaza,0,0,0,0,0,0,0,0,0,...,253922,256461,259133,262017,264395,265897,268132,270856,272767,274690
189,Yemen,0,0,0,0,0,0,0,0,0,...,4881,4975,5047,5133,5233,5276,5357,5446,5507,5582
190,Zambia,0,0,0,0,0,0,0,0,0,...,89009,89071,89386,89592,89783,89918,90029,90064,90218,90389


In [104]:
cols=df1[df1.columns[1:-1]]
df2=df1.drop(cols,axis=1)

In [105]:
df2

,Country/Region,4/14/21
0,Afghanistan,57492
1,Albania,128959
2,Algeria,118975
3,Andorra,12641
4,Angola,23841
...,...,...
187,Vietnam,2733
188,West Bank and Gaza,274690
189,Yemen,5582
190,Zambia,90389


In [107]:
df=df.drop('GDP (BILLIONS)',axis=1)
df

,COUNTRY,CODE
0,Afghanistan,AFG
1,Albania,ALB
2,Algeria,DZA
3,American Samoa,ASM
4,Andorra,AND
...,...,...
217,Virgin Islands,VGB
218,West Bank,WBG
219,Yemen,YEM
220,Zambia,ZMB


In [108]:
df3=pd.merge(df2,df,how='left',left_on=['Country/Region'],right_on=['COUNTRY'])
df3

,Country/Region,4/14/21,COUNTRY,CODE
0,Afghanistan,57492,Afghanistan,AFG
1,Albania,128959,Albania,ALB
2,Algeria,118975,Algeria,DZA
3,Andorra,12641,Andorra,AND
4,Angola,23841,Angola,AGO
...,...,...,...,...
187,Vietnam,2733,Vietnam,VNM
188,West Bank and Gaza,274690,NaN,NaN
189,Yemen,5582,Yemen,YEM
190,Zambia,90389,Zambia,ZMB


In [ ]:
df['your column name'].isnull().values.any()
df['your column name'].isnull().sum()